In [ ]:
# ! pip install langchain_openai
# ! pip install load_dotenv
# ! pip install tiktoken
# ! pip install langchain_huggingface
# ! pip install openpyxl
# ! pip install chromadb
# ! pip install rank_bm25

In [19]:
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import os

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEndpoint
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI

from openpyxl import load_workbook
from typing import Dict, List, Optional
from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader

In [20]:
load_dotenv()
api_key = os.environ.get('Openai_key')

사용할 데이터 불러오기

In [21]:
# class XLSXLoader(BaseLoader):
#     """Loads an XLSX file into a list of documents from specified sheets.

#     Each document represents one row of the XLSX file. Every row is converted into a
#     key/value pair and outputted to a new line in the document's page_content, including
#     the sheet name.

#     The source for each document loaded from xlsx is set to the value of the
#     'file_path' argument for all documents by default.
#     You can override this by setting the 'source_column' argument to the
#     name of a column in the XLSX file.
#     The source of each document will then be set to the value of the column
#     with the name specified in 'source_column'.

#     Output Example:
#         .. code-block:: txt

#             sheet: SheetName
#             column1: value1
#             column2: None
#             column3: value3
#     """

#     docs = []

#     def __init__(
#             self,
#             file_path: str,
#             source_column: Optional[str] = None,
#             encoding: Optional[str] = None,
#             exclude_columns: Optional[List[str]] = None,  # 열을 제외할 목록
#             target_sheets: Optional[List[str]] = None  # 불러올 시트 목록
#     ):
#         self.file_path = file_path
#         self.source_column = source_column
#         self.encoding = encoding
#         self.exclude_columns = exclude_columns if exclude_columns else []
#         self.target_sheets = target_sheets if target_sheets else []

#     def load(self) -> List[Document]:
#         docs = []

#         wb = load_workbook(filename=self.file_path, read_only=True, data_only=True)

#         # 특정 시트를 처리
#         sheets_to_process = self.target_sheets if self.target_sheets else wb.sheetnames[1:]


#         for sheet_name in sheets_to_process:
#             if sheet_name in wb.sheetnames:
#                 ws = wb[sheet_name]
#                 headers = [cell.value for cell in ws[1]]
#                 headers = [header if header is not None else '' for header in headers]  # None을 빈 문자열로 변환

#                 # 이전 행의 값을 저장하기 위한 리스트
#                 previous_row = [None] * len(headers)

#                 for i, row in enumerate(ws.iter_rows(min_row=2)):
#                     row_values = [cell.value for cell in row]

#                     # None 값을 이전 행의 값으로 대체, 제외할 열을 처리
#                     for j, (value, header) in enumerate(zip(row_values, headers)):
#                         if value is None and header not in self.exclude_columns:
#                             row_values[j] = previous_row[j]

#                         # '단종 여부' 열 처리
#                         if header == '단종 여부' and value is None:
#                             row_values[j] = '해당없음'

#                         # '구분' 열 처리
#                         if header == '구분':
#                             if value is None:
#                                 row_values[j] = previous_row[j]
#                             if 'ODM' in (row_values[j] or ''):
#                                 row_values[j] = 'ODM'
#                             else:
#                                 row_values[j] = 'OEM'

#                         # '런칭여부' 열 처리
#                         if header == '런칭여부':
#                             if value is None:
#                                 row_values[j] = previous_row[j]
#                             # else 문을 제거하여 중복되는 처리를 피함

#                     row_dict = dict(zip(headers, row_values))


#                     # # 시트 이름과 None 값을 처리하여 'None' 문자열로 변환
#                     # content_lines = [f"sheet: {sheet_name}"]  # 시트 이름 추가
#                     # content_lines.extend(
#                     #     f"{k.strip()}: {str(v) if v is not None else 'None'}"
#                     #     for k, v in row_dict.items()
#                     #     if k.strip() != ''  # 빈 헤더 무시
#                     # )
#                     # content = "\n".join(content_lines)

#                     if self.source_column is not None:
#                         source = row_dict.get(self.source_column, 'None')  # source_column이 없는 경우 'None' 사용
#                     else:
#                         source = self.file_path

#                     lines = f"""제품의 이름은 {row_dict['제품명']}입니다. 해당 제품의 채널, 고객사는 {row_dict['채널, 고객사']}이며,단종 여부는 {row_dict['단종 여부']}입니다. 제품의 런칭여부는 {row_dict['런칭여부']}입니다. 제품은 {row_dict['구분']}으로 구분되어있으며, 제품의 사용원료는 {row_dict['사용원료']}입니다. 제품의 컨셉은 {row_dict['컨셉']}입니다."""


#                     metadata = {"source": source, "sheet": sheet_name}
#                     row_dict = Document(page_content=lines, metadata=metadata)
#                     docs.append(row_dict)

#                     # 현재 행의 값을 이전 행으로 저장
#                     previous_row = row_values

#         return docs


In [22]:
class XLSXLoader(BaseLoader):
    """Loads an XLSX file into a list of documents from specified sheets.

    Each document represents one row of the XLSX file. Every row is converted into a
    key/value pair and outputted to a new line in the document's page_content, including
    the sheet name.

    The source for each document loaded from xlsx is set to the value of the
    'file_path' argument for all documents by default.
    You can override this by setting the 'source_column' argument to the
    name of a column in the XLSX file.
    The source of each document will then be set to the value of the column
    with the name specified in 'source_column'.

    Output Example:
        .. code-block:: txt

            sheet: SheetName
            column1: value1
            column2: None
            column3: value3
    """

    docs = []

    def __init__(
            self,
            file_path: str,
            source_column: Optional[str] = None,
            encoding: Optional[str] = None,
            exclude_columns: Optional[List[str]] = None,  # 열을 제외할 목록
            target_sheets: Optional[List[str]] = None  # 불러올 시트 목록
    ):
        self.file_path = file_path
        self.source_column = source_column
        self.encoding = encoding
        self.exclude_columns = exclude_columns if exclude_columns else []
        self.target_sheets = target_sheets if target_sheets else []

    def load(self) -> List[Document]:
        docs = []

        wb = load_workbook(filename=self.file_path, read_only=True, data_only=True)

        # 특정 시트를 처리
        sheets_to_process = self.target_sheets if self.target_sheets else wb.sheetnames[1:]


        for sheet_name in sheets_to_process:
            if sheet_name in wb.sheetnames:
                ws = wb[sheet_name]
                headers = [cell.value for cell in ws[1]]
                headers = [header if header is not None else '' for header in headers]  # None을 빈 문자열로 변환

                # 이전 행의 값을 저장하기 위한 리스트
                previous_row = [None] * len(headers)

                for i, row in enumerate(ws.iter_rows(min_row=2)):
                    row_values = [cell.value for cell in row]

                    # None 값을 이전 행의 값으로 대체, 제외할 열을 처리
                    for j, (value, header) in enumerate(zip(row_values, headers)):
                        if value is None and header not in self.exclude_columns:
                            row_values[j] = previous_row[j]

                        # '단종 여부' 열 처리
                        if header == '단종 여부' and value is None:
                            row_values[j] = '미단종'

                        # '구분' 열 처리
                        if header == '구분':
                            if value is None:
                                row_values[j] = previous_row[j]
                            if 'ODM' in (row_values[j] or ''):
                                row_values[j] = 'ODM'
                            else:
                                row_values[j] = 'OEM'

                        # '런칭여부' 열 처리
                        if header == '런칭여부':
                            if value is None:
                                row_values[j] = previous_row[j]
                            # else 문을 제거하여 중복되는 처리를 피함

                    row_dict = dict(zip(headers, row_values))


                    # # 시트 이름과 None 값을 처리하여 'None' 문자열로 변환
                    # content_lines = [f"sheet: {sheet_name}"]  # 시트 이름 추가
                    # content_lines.extend(
                    #     f"{k.strip()}: {str(v) if v is not None else 'None'}"
                    #     for k, v in row_dict.items()
                    #     if k.strip() != ''  # 빈 헤더 무시
                    # )
                    # content = "\n".join(content_lines)

                    # if self.source_column is not None:
                    #     source = row_dict.get(self.source_column, 'None')  # source_column이 없는 경우 'None' 사용
                    # else:
                    #     source = self.file_path

                    # lines = f"""제품의 이름은 {row_dict['제품명']}입니다. 해당 제품의 채널, 고객사는 {row_dict['채널, 고객사']}이며,단종 여부는 {row_dict['단종 여부']}입니다. 제품의 런칭여부는 {row_dict['런칭여부']}입니다. 제품은 {row_dict['구분']}으로 구분되어있으며, 제품의 사용원료는 {row_dict['사용원료']}입니다. 제품의 컨셉은 {row_dict['컨셉']}입니다."""


                    # metadata = {"source": source, "sheet": sheet_name}
                    # row_dict = Document(page_content=lines, metadata=metadata)
                    docs.append(row_dict)

                    # 현재 행의 값을 이전 행으로 저장
                    previous_row = row_values

        return docs


In [23]:
# 파일 불러오기
loader = XLSXLoader(
    file_path="C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/data/년도별 신제품 리스트_냉장-240425.xlsx",
    source_column="제품명",
    exclude_columns=["단종 여부"],  # 제외할 열 지정
    target_sheets=["2023"]  # 불러올 시트 지정
)

docs = loader.load()

In [24]:
docs

[{'구분': 'OEM',
  '채널, 고객사': '외식자재',
  '제품명': '멜팅피스 핫도그용소시지(오리지널)',
  '단종 여부': '미단종',
  '런칭여부': '런칭',
  '사용원료': '후지, 지방',
  '컨셉': '고기씹는 거친 식감과 육즙이 풍부한 핫도그용 소시지(멜팅피스)',
  '': None},
 {'구분': 'OEM',
  '채널, 고객사': '외식자재',
  '제품명': '멜팅피스 핫도그용소시지(치즈)',
  '단종 여부': '미단종',
  '런칭여부': '런칭',
  '사용원료': '후지, 지방',
  '컨셉': '육 식감이 살아있고 다즙한 콘도그용 치즈 소시지',
  '': None},
 {'구분': 'OEM',
  '채널, 고객사': '외식자재',
  '제품명': '멜팅피스 핫도그용소시지(치즈&할라피뇨)',
  '단종 여부': '미단종',
  '런칭여부': '런칭',
  '사용원료': '후지, 지방',
  '컨셉': '육 식감이 살아있고 다즙한 콘도그용 치즈&할라피뇨 소시지',
  '': None},
 {'구분': 'OEM',
  '채널, 고객사': '학교급식',
  '제품명': '초당옥수수핫도그용소시지',
  '단종 여부': '미단종',
  '런칭여부': '런칭',
  '사용원료': '후지, 지방',
  '컨셉': '육즙이 풍부한 학교급식용(아질산/MSG 제외) 미니 소시지(#21콜라겐 케이싱, 20g) ',
  '': None},
 {'구분': 'OEM',
  '채널, 고객사': '학교급식',
  '제품명': '트리플슬라이스소시지',
  '단종 여부': '미단종',
  '런칭여부': '런칭',
  '사용원료': '후지, 지방',
  '컨셉': '모듬슬라이스소시지 리뉴얼(갈비맛슬라이스소시지, 할라피뇨슬라이스소시지 신규 개발/적용)',
  '': None},
 {'구분': 'OEM',
  '채널, 고객사': '외식자재',
  '제품명': '본레스햄 슬라이스',
  '단종 여부': '미단종',
  '런칭여부': '런칭',
  '

In [40]:
documents = []

for doc in docs:
    line = f"""제품의 이름은 {doc['제품명']}입니다. 해당 제품의 채널, 고객사는 {doc['채널, 고객사']}이며,단종 여부는 {doc['단종 여부']}입니다. 제품의 런칭여부는 {doc['런칭여부']}입니다. 제품은 {doc['구분']}으로 구분되어있으며, 제품의 사용원료는 {doc['사용원료']}입니다. 제품의 컨셉은 {doc['컨셉']}입니다."""
    documents.append(line)

In [41]:
documents

['제품의 이름은 멜팅피스 핫도그용소시지(오리지널)입니다. 해당 제품의 채널, 고객사는 외식자재이며,단종 여부는 미단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 OEM으로 구분되어있으며, 제품의 사용원료는 후지, 지방입니다. 제품의 컨셉은 고기씹는 거친 식감과 육즙이 풍부한 핫도그용 소시지(멜팅피스)입니다.',
 '제품의 이름은 멜팅피스 핫도그용소시지(치즈)입니다. 해당 제품의 채널, 고객사는 외식자재이며,단종 여부는 미단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 OEM으로 구분되어있으며, 제품의 사용원료는 후지, 지방입니다. 제품의 컨셉은 육 식감이 살아있고 다즙한 콘도그용 치즈 소시지입니다.',
 '제품의 이름은 멜팅피스 핫도그용소시지(치즈&할라피뇨)입니다. 해당 제품의 채널, 고객사는 외식자재이며,단종 여부는 미단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 OEM으로 구분되어있으며, 제품의 사용원료는 후지, 지방입니다. 제품의 컨셉은 육 식감이 살아있고 다즙한 콘도그용 치즈&할라피뇨 소시지입니다.',
 '제품의 이름은 초당옥수수핫도그용소시지입니다. 해당 제품의 채널, 고객사는 학교급식이며,단종 여부는 미단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 OEM으로 구분되어있으며, 제품의 사용원료는 후지, 지방입니다. 제품의 컨셉은 육즙이 풍부한 학교급식용(아질산/MSG 제외) 미니 소시지(#21콜라겐 케이싱, 20g) 입니다.',
 '제품의 이름은 트리플슬라이스소시지입니다. 해당 제품의 채널, 고객사는 학교급식이며,단종 여부는 미단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 OEM으로 구분되어있으며, 제품의 사용원료는 후지, 지방입니다. 제품의 컨셉은 모듬슬라이스소시지 리뉴얼(갈비맛슬라이스소시지, 할라피뇨슬라이스소시지 신규 개발/적용)입니다.',
 '제품의 이름은 본레스햄 슬라이스입니다. 해당 제품의 채널, 고객사는 외식자재이며,단종 여부는 미단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 OEM으로 구분되어있으며, 제품의 사용원료는 목살

In [27]:
# # 토크나이징
# tokenizer = tiktoken.get_encoding("cl100k_base")

# def tiktoken_len(text):
#     tokens = tokenizer.encode(text)
#     return len(tokens)

In [28]:
# # chunking
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10, length_function=tiktoken_len)
# texts = text_splitter.split_documents(docs)

In [165]:
# # 벡터화에 사용할 모델(huggingFaceEmbeddings)
# model_name ="jhgan/ko-sbert-nli"
# # model_kwargs = {'device': 'cuda'} # gpu 사용
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': True}
# hf = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs
# )

c:\Users\SUNJIN\anaconda3\envs\intern\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\SUNJIN\anaconda3\envs\intern\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [178]:
# # openaiEmbeddings
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small", api_key=api_key)

db = Chroma.from_texts(
    documents, 
    embeddings_model,
    collection_name = 'history',
    persist_directory = './db/chromadb',
    collection_metadata = {'hnsw:space': 'cosine'}, # l2 is the default
)

db

모델

In [166]:
# DB에 넣고 리트리버 구성
# docsearch = Chroma.from_documents(docs, hf) # chunking 을 안하면서 texts -> documents로
# retriever = db.as_retriever(    search_type='mmr',
#     search_kwargs={'k': 5, 'lambda_mult': 0.15})

In [179]:
retriever = db.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={'score_threshold': 0.3}
)

In [181]:
retriever = db.as_retriever(
    search_type='mmr'
)

In [182]:
d = retriever.get_relevant_documents("닭으로 만들어진 제품은?")
d

[Document(page_content='제품의 이름은 훈제닭다리스테이크(장각)입니다. 해당 제품의 채널, 고객사는 군납이며,단종 여부는 미단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 장각입니다. 제품의 컨셉은 선진스모크치킨장각 외주 OEM 이관 제품 (주원산오리)입니다.'),
 Document(page_content='제품의 이름은 선진 허브갈릭닭가슴살소시지입니다. 해당 제품의 채널, 고객사는 제이브로(군납)이며,단종 여부는 해당없음입니다. 제품의 런칭여부는 런칭입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 종계 가슴살입니다. 제품의 컨셉은 계육 함량 80% 이상의 퍽퍽하지 않은 부드럽고 탄력있는 닭가슴살 소시지입니다.'),
 Document(page_content='제품의 이름은 한돈 양념갈비입니다. 해당 제품의 채널, 고객사는 담꾹이며,단종 여부는 미단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 갈비입니다. 제품의 컨셉은 담꾹 메뉴시연회용 갈비찜 요청(돈육, 우육) / LA 돈갈비를 사용한 한돈 양념갈비 확정입니다.'),
 Document(page_content='제품의 이름은 선진 닭가슴살슬라이스햄입니다. 해당 제품의 채널, 고객사는 군납이며,단종 여부는 미단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 육계가슴살입니다. 제품의 컨셉은 군납용 트레이 포장된 닭가슴살 슬라이스햄입니다.')]

In [180]:
d = retriever.get_relevant_documents("닭으로 만들어진 제품은?")
d

[Document(page_content='제품의 이름은 훈제닭다리스테이크(장각)입니다. 해당 제품의 채널, 고객사는 군납이며,단종 여부는 미단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 장각입니다. 제품의 컨셉은 선진스모크치킨장각 외주 OEM 이관 제품 (주원산오리)입니다.'),
 Document(page_content='제품의 이름은 훈제닭다리스테이크(장각)입니다. 해당 제품의 채널, 고객사는 군납이며,단종 여부는 해당없음입니다. 제품의 런칭여부는 런칭입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 장각입니다. 제품의 컨셉은 선진스모크치킨장각 외주 OEM 이관 제품 (주원산오리)입니다.'),
 Document(page_content='제품의 이름은 훈제닭다리스테이크(장각)입니다. 해당 제품의 채널, 고객사는 군납이며,단종 여부는 해당없음입니다. 제품의 런칭여부는 런칭입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 장각입니다. 제품의 컨셉은 선진스모크치킨장각 외주 OEM 이관 제품 (주원산오리)입니다.'),
 Document(page_content='제품의 이름은 훈제닭다리스테이크(장각)입니다. 해당 제품의 채널, 고객사는 군납이며,단종 여부는 해당없음입니다. 제품의 런칭여부는 런칭입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 장각입니다. 제품의 컨셉은 선진스모크치킨장각 외주 OEM 이관 제품 (주원산오리)입니다.')]

In [159]:
from langchain.retrievers import BM25Retriever

BM25_retriever = BM25Retriever.from_texts(documents, k = 5)
Chroma_retriever = db.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={'score_threshold': 0.4}
)

In [160]:
from langchain.retrievers import EnsembleRetriever

In [161]:
ensemble_retriever = EnsembleRetriever(
                    retrievers = [BM25_retriever,Chroma_retriever]
                    , weight = {0.5,0.5})

In [162]:
docs = ensemble_retriever.invoke("닭으로 만들어진 제품은?")
docs

[Document(page_content='제품의 이름은 말이어묵용 소시지입니다. 해당 제품의 채널, 고객사는 풀무원이며,단종 여부는 미단종입니다. 제품의 런칭여부는 뱅킹화입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 후지, 지방입니다. 제품의 컨셉은 말이어묵 상품 안에 들어가는 소시지 제품입니다.'),
 Document(page_content='진팸[부대찌개용]은  혜자도시락 부대찌개볶음 도시락 구성품.  해당 제품은 GS25 타겟으로 한다. 현재 단종 되었으며 런칭 제품이다. 제품은 ODM으로 구분되어있으며, 후지, 지방로 만들어졌다.'),
 Document(page_content='제품의 이름은 한우언양불고기입니다. 해당 제품의 채널, 고객사는 반올림피자샵이며,단종 여부는 미단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 한우입니다. 제품의 컨셉은 한우 목살을 활용한 피자 토핑용 제품입니다.'),
 Document(page_content='한우언양불고기은  한우 목살을 활용한 피자 토핑용 제품.  해당 제품은 반올림피자샵 타겟으로 한다. 현재 미단종 되었으며 런칭 제품이다. 제품은 ODM으로 구분되어있으며, 한우로 만들어졌다.'),
 Document(page_content='제품의 이름은 라구소스토핑입니다. 해당 제품의 채널, 고객사는 풀무원이며,단종 여부는 미단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 후지입니다. 제품의 컨셉은 볼로네제 풍미의 소스가 혼합된 냉동 피자용 돈육 토핑입니다.'),
 Document(page_content='제품의 이름은 죠쏘용소시지입니다. 해당 제품의 채널, 고객사는 GS25이며,단종 여부는 미단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 후지, 지방입니다. 제품의 컨셉은 떡볶이 국물의 맛에 어우러지는 칼집이 난 연한 훈연이 들어간 미니소시지입니다.'),
 Docum

In [101]:
# DB에 넣고 리트리버 구성
# docsearch = Chroma.from_documents(docs, hf) # chunking 을 안하면서 texts -> documents로
retriever = docsearch.as_retriever(    search_type='similarity_score_threshold',
    search_kwargs={'score_threshold': 0.1})

In [103]:
d = retriever.get_relevant_documents("목살 제품")
print(f"문서의 개수: {len(d)}")
d

문서의 개수: 4


[Document(metadata={'sheet': '2023', 'source': '돈목살 양념육 2종'}, page_content='제품의 이름은 돈목살 양념육 2종입니다. 해당 제품의 채널, 고객사는 후레시스이며,단종 여부는 해당없음입니다. 제품의 런칭여부는 뱅킹화입니다. 제품은 OEM으로 구분되어있으며, 제품의 사용원료는 목살입니다. 제품의 컨셉은 돈목살을 활용한 2가지 종류의 도시락용 비가열 양념육 제품 (간장/고추장)\n고객사 지속 샘플링 중입니다.'),
 Document(metadata={'sheet': '2023', 'source': '본레스햄 슬라이스'}, page_content='제품의 이름은 본레스햄 슬라이스입니다. 해당 제품의 채널, 고객사는 성심당이며,단종 여부는 단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 목살입니다. 제품의 컨셉은 육괴 입자를 살린 재구성 원형 슬라이스햄입니다.'),
 Document(metadata={'sheet': '2023', 'source': '초당옥수수핫도그용소시지'}, page_content='제품의 이름은 초당옥수수핫도그용소시지입니다. 해당 제품의 채널, 고객사는 학교급식이며,단종 여부는 해당없음입니다. 제품의 런칭여부는 런칭입니다. 제품은 OEM으로 구분되어있으며, 제품의 사용원료는 후지, 지방입니다. 제품의 컨셉은 육즙이 풍부한 학교급식용(아질산/MSG 제외) 미니 소시지(#21콜라겐 케이싱, 20g) 입니다.'),
 Document(metadata={'sheet': '2023', 'source': '빅킬바사소시지'}, page_content='제품의 이름은 빅킬바사소시지입니다. 해당 제품의 채널, 고객사는 GS25이며,단종 여부는 단종입니다. 제품의 런칭여부는 런칭입니다. 제품은 ODM으로 구분되어있으며, 제품의 사용원료는 후지, 지방입니다. 제품의 컨셉은 돈장 활용 말발굽형태의 냉장 소시지 제품입니다.')]

In [ ]:
llm = ChatOpenAI(base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
    model="xionic-ko-llama-3-70b",
    temperature=0.7)

In [ ]:
# openai = ChatOpenAI(model_name="gpt-4o-mini",
#                     streaming=True, callbacks=[StreamingStdOutCallbackHandler()],
#                     temperature = 0,
#                     max_tokens = 500,
#                     api_key=api_key)

In [ ]:
contextualize_q_system_prompt = (
      "당신은 회사 제품을 잘 알고 있는 마케딩 디렉터입니다."
      "답변은 사용자가 질문한 언어와 같은 언어를 사용하세요."
      "다음 질문에 대해 주어진 문맥을 사용하여 상세하고 정확한 답변을 제공해주세요. "
      "관련 문맥 부분을 참조하여 답변을 작성하세요.\n\n"
      "{context}"
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        "{context}",
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [ ]:
system_prompt = (
    """
    You MUST Answer in Korean.
    당신은 회사 제품을 잘 알고 있는 신제품 개발 팀장입니다.
    새로운 제품을 출시하기 위해 과거 신제품에 대한 정보를 모두 알고 있으며 팀원들이 특정 제품에 대해 물어볼 때 모두 대답할 수 있어야합니다.
    물어본 특성에 해당하는 제품은 하나도 빠짐없이 대답해줘야 합니다.

    예를 들어, "2023년도에 나온 제품들 중 단종된 제품을 전부 말해줘" 라고 물어본다면,

    2023년도에 기획된 제품 30건 중 단종 된 제품은 총 ""가지가 있습니다.
    "" , "", "" 입니다.
    다른 제품의 경우 특이사항이 없으나 ""과 "" 의 경우 ""을 이유로 조기단종하였습니다.

    특이사항이 있는 경우 더 말해줘야해

    {context}
    
    이건 사용자의 질문입니다. {input}
    """
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        "{context}",
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
conversational_rag_chain.invoke(
    {"input": "GS25에 납품하는 제품 모두 알려줘"},
    config={"configurable": {"session_id": "abc123"}},
)
